# 📊 Évolution des Capacités - Pitié-Salpêtrière (2019-2024)

Ce notebook analyse l'évolution des capacités hospitalières de Pitié-Salpêtrière sur 6 ans, basé sur les données **officielles SAE (DREES)**.

**Source:** Statistique Annuelle des Établissements de Santé  
**FINESS:** 750100125  
**Établissement:** GHU APHP - Site Pitié-Salpêtrière

## Points clés à observer:
- 🦠 **2020**: Impact COVID (hausse réa, baisse activité programmée)
- 📉 **2022**: Restructurations AP-HP (fermetures de lits)
- 📈 **2019-2024**: Virage ambulatoire (+15% places)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

DATA_DIR = Path('../data')

print("✅ Configuration chargée")

## 1. Chargement des données réelles SAE

In [ ]:
# Charger les données réelles extraites des SAE
df = pd.read_csv(DATA_DIR / 'evolution_reelle_pitie_2019_2024.csv')

print("📊 DONNÉES SAE RÉELLES - PITIÉ-SALPÊTRIÈRE")
print("="*60)
print(f"Période: {df['annee'].min()} - {df['annee'].max()}")
print(f"Colonnes: {len(df.columns)}")

df

## 2. Évolution des lits MCO (Médecine-Chirurgie-Obstétrique)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1: Évolution par type
ax1 = axes[0]
ax1.plot(df['annee'], df['lits_medecine'], 'o-', label='Médecine', linewidth=2, markersize=8, color='#2ecc71')
ax1.plot(df['annee'], df['lits_chirurgie'], 's-', label='Chirurgie', linewidth=2, markersize=8, color='#3498db')
ax1.plot(df['annee'], df['lits_obstetrique'], '^-', label='Obstétrique', linewidth=2, markersize=8, color='#e74c3c')

# Zone COVID
ax1.axvspan(2019.5, 2021.5, alpha=0.2, color='red', label='Période COVID')
# Zone restructurations
ax1.axvspan(2021.5, 2022.5, alpha=0.2, color='orange', label='Restructurations AP-HP')

ax1.set_xlabel('Année', fontsize=12)
ax1.set_ylabel('Nombre de lits', fontsize=12)
ax1.set_title('Évolution des lits par spécialité MCO', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Graphique 2: Total MCO avec annotations
ax2 = axes[1]
colors = ['#3498db' if y not in [2020, 2022] else ('#e74c3c' if y == 2020 else '#f39c12') for y in df['annee']]
bars = ax2.bar(df['annee'], df['lits_mco_total'], color=colors, alpha=0.8, edgecolor='black')

# Annotations
for i, (bar, val, year) in enumerate(zip(bars, df['lits_mco_total'], df['annee'])):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20, 
             f'{int(val)}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    if year == 2020:
        ax2.annotate('🦠 COVID', (year, val), xytext=(year, val+100), fontsize=9, ha='center')
    elif year == 2022:
        ax2.annotate('📉 -16%', (year, val), xytext=(year, val+100), fontsize=9, ha='center', color='red')

ax2.set_xlabel('Année', fontsize=12)
ax2.set_ylabel('Nombre de lits', fontsize=12)
ax2.set_title('Total lits MCO - Impact visible', fontsize=14, fontweight='bold')
ax2.set_ylim(0, df['lits_mco_total'].max() * 1.2)

plt.tight_layout()
plt.savefig('../data/graphs/evolution_lits_mco_reel.png', dpi=150, bbox_inches='tight')
plt.show()

# Stats
evolution_mco = ((df['lits_mco_total'].iloc[-1] - df['lits_mco_total'].iloc[0]) / df['lits_mco_total'].iloc[0]) * 100
print(f"\n📈 Évolution MCO 2019→2024: {evolution_mco:+.1f}%")
print(f"   Perte de {int(df['lits_mco_total'].iloc[0] - df['lits_mco_total'].iloc[-1])} lits")

## 3. Impact COVID sur la Réanimation

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1: Lits de réanimation
ax1 = axes[0]
colors_rea = ['#e74c3c' if y == 2020 else '#3498db' for y in df['annee']]
bars = ax1.bar(df['annee'], df['lits_reanimation'], color=colors_rea, alpha=0.8, edgecolor='black')

for bar, val, year in zip(bars, df['lits_reanimation'], df['annee']):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
             f'{int(val)}', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax1.annotate('🦠 PIC COVID\n+14%', (2020, 117), xytext=(2020.3, 125), 
             fontsize=10, color='red', fontweight='bold',
             arrowprops=dict(arrowstyle='->', color='red'))

ax1.set_xlabel('Année', fontsize=12)
ax1.set_ylabel('Lits de réanimation', fontsize=12)
ax1.set_title('Lits de Réanimation - Impact COVID visible', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 140)

# Graphique 2: Journées et séjours réa
ax2 = axes[1]
ax2.bar(df['annee'] - 0.2, df['journees_rea']/1000, width=0.4, label='Journées (milliers)', color='#e74c3c', alpha=0.7)
ax2.bar(df['annee'] + 0.2, df['sejours_rea']/100, width=0.4, label='Séjours (centaines)', color='#3498db', alpha=0.7)

ax2.axvspan(2019.5, 2021.5, alpha=0.1, color='red')
ax2.set_xlabel('Année', fontsize=12)
ax2.set_ylabel('Activité', fontsize=12)
ax2.set_title('Activité Réanimation (journées et séjours)', fontsize=14, fontweight='bold')
ax2.legend()

plt.tight_layout()
plt.savefig('../data/graphs/evolution_reanimation_covid.png', dpi=150, bbox_inches='tight')
plt.show()

# Stats COVID
print("\n🦠 IMPACT COVID (2019 → 2020):")
print(f"   Lits réa: {df[df['annee']==2019]['lits_reanimation'].values[0]} → {df[df['annee']==2020]['lits_reanimation'].values[0]} (+{((117-103)/103*100):.0f}%)")
print(f"   Journées réa: {df[df['annee']==2019]['journees_rea'].values[0]:,} → {df[df['annee']==2020]['journees_rea'].values[0]:,}")

## 4. Virage Ambulatoire

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Ratio ambulatoire / lits
ratio = df['places_ambulatoire'] / df['lits_mco_total'] * 100

ax.fill_between(df['annee'], 0, ratio, alpha=0.3, color='#2ecc71')
ax.plot(df['annee'], ratio, 'o-', color='#2ecc71', linewidth=3, markersize=10)

for year, r, places in zip(df['annee'], ratio, df['places_ambulatoire']):
    ax.annotate(f'{r:.1f}%\n({int(places)} places)', (year, r), 
                textcoords='offset points', xytext=(0, 15), ha='center', fontsize=9)

ax.set_xlabel('Année', fontsize=12)
ax.set_ylabel('Ratio Places Ambulatoire / Lits MCO (%)', fontsize=12)
ax.set_title('Virage Ambulatoire - Pitié-Salpêtrière', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/graphs/virage_ambulatoire.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📈 VIRAGE AMBULATOIRE:")
print(f"   2019: {df[df['annee']==2019]['places_ambulatoire'].values[0]} places ({ratio.iloc[0]:.1f}% des lits)")
print(f"   2024: {df[df['annee']==2024]['places_ambulatoire'].values[0]} places ({ratio.iloc[-1]:.1f}% des lits)")
print(f"   Évolution: +{int(df[df['annee']==2024]['places_ambulatoire'].values[0] - df[df['annee']==2019]['places_ambulatoire'].values[0])} places")

## 5. Taux d'occupation MCO

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

colors_occ = ['#e74c3c' if t < 75 else ('#f39c12' if t < 80 else '#2ecc71') for t in df['taux_occupation_mco']]
bars = ax.bar(df['annee'], df['taux_occupation_mco'], color=colors_occ, alpha=0.8, edgecolor='black')

# Ligne cible 85%
ax.axhline(85, color='red', linestyle='--', linewidth=2, label='Cible nationale (85%)')
ax.axhline(75, color='orange', linestyle=':', linewidth=1, label='Seuil critique (75%)')

for bar, val, year in zip(bars, df['taux_occupation_mco'], df['annee']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{val:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Annotation COVID
ax.annotate('🦠 Chute activité\n(déprogrammation)', (2020, 68.8), xytext=(2020.5, 60), 
             fontsize=9, color='red',
             arrowprops=dict(arrowstyle='->', color='red'))

ax.set_xlabel('Année', fontsize=12)
ax.set_ylabel('Taux d\'occupation (%)', fontsize=12)
ax.set_title('Taux d\'occupation MCO - Impact COVID et reprise', fontsize=14, fontweight='bold')
ax.set_ylim(50, 95)
ax.legend(loc='lower right')

plt.tight_layout()
plt.savefig('../data/graphs/taux_occupation_mco.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 TAUX D'OCCUPATION MCO:")
print(f"   2019 (pré-COVID): {df[df['annee']==2019]['taux_occupation_mco'].values[0]:.1f}%")
print(f"   2020 (COVID):     {df[df['annee']==2020]['taux_occupation_mco'].values[0]:.1f}% ← Déprogrammation")
print(f"   2024 (actuel):    {df[df['annee']==2024]['taux_occupation_mco'].values[0]:.1f}%")

## 6. Synthèse globale

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Lits MCO
ax1 = axes[0, 0]
ax1.fill_between(df['annee'], df['lits_mco_total'], alpha=0.3, color='#3498db')
ax1.plot(df['annee'], df['lits_mco_total'], 'o-', color='#3498db', linewidth=2, markersize=8)
ax1.axvspan(2019.5, 2021.5, alpha=0.1, color='red')
ax1.set_title('Lits MCO (-19%)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Lits')

# 2. Réanimation
ax2 = axes[0, 1]
ax2.fill_between(df['annee'], df['lits_reanimation'], alpha=0.3, color='#e74c3c')
ax2.plot(df['annee'], df['lits_reanimation'], 'o-', color='#e74c3c', linewidth=2, markersize=8)
ax2.axvspan(2019.5, 2021.5, alpha=0.1, color='red')
ax2.set_title('Lits Réanimation (pic COVID +14%)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Lits')

# 3. SSR
ax3 = axes[1, 0]
ax3.fill_between(df['annee'], df['lits_ssr'], alpha=0.3, color='#9b59b6')
ax3.plot(df['annee'], df['lits_ssr'], 'o-', color='#9b59b6', linewidth=2, markersize=8)
ax3.axvspan(2021.5, 2022.5, alpha=0.1, color='orange')
ax3.set_title('Lits SSR (-28%)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Lits')
ax3.set_xlabel('Année')

# 4. Ambulatoire
ax4 = axes[1, 1]
ax4.fill_between(df['annee'], df['places_ambulatoire'], alpha=0.3, color='#2ecc71')
ax4.plot(df['annee'], df['places_ambulatoire'], 'o-', color='#2ecc71', linewidth=2, markersize=8)
ax4.set_title('Places Ambulatoire (+15%)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Places')
ax4.set_xlabel('Année')

for ax in axes.flat:
    ax.grid(True, alpha=0.3)

plt.suptitle('Évolution Pitié-Salpêtrière 2019-2024 (Données SAE réelles)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../data/graphs/synthese_evolution_reelle.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Tableau de synthèse

In [ ]:
# Créer tableau de synthèse
synthese = df[['annee', 'lits_mco_total', 'lits_reanimation', 'lits_ssr', 'places_ambulatoire', 'taux_occupation_mco']].copy()
synthese.columns = ['Année', 'Lits MCO', 'Lits Réa', 'Lits SSR', 'Places Ambul.', 'Taux Occ. (%)']
synthese = synthese.set_index('Année')

# Calculer évolutions
print("\n" + "="*70)
print("SYNTHÈSE ÉVOLUTION PITIÉ-SALPÊTRIÈRE (SAE 2019-2024)")
print("="*70)
display(synthese)

print("\n📊 ÉVOLUTIONS CLÉS:")
for col in ['Lits MCO', 'Lits Réa', 'Lits SSR', 'Places Ambul.']:
    first = synthese[col].iloc[0]
    last = synthese[col].iloc[-1]
    evol = ((last - first) / first) * 100
    print(f"   {col}: {int(first)} → {int(last)} ({evol:+.1f}%)")

## 8. Contexte et Interprétation

In [ ]:
print("""
📋 CONTEXTE ET INTERPRÉTATION
{'='*60}

🦠 PÉRIODE COVID (2020-2021):
   • Pic réanimation: +14% de lits (103 → 117)
   • Chute activité programmée: -16% séjours médecine
   • Taux occupation MCO: 79% → 69% (déprogrammation)
   • Reconversion de lits MCO vers soins critiques

📉 RESTRUCTURATIONS AP-HP (2022):
   • Fermeture massive: -232 lits MCO (-16%)
   • Réduction SSR: -28 lits (-27%)
   • Plan d'économies et réorganisation des GHU
   • Contexte de crise des urgences (été 2022)

📈 TENDANCES STRUCTURELLES:
   • Virage ambulatoire: +15% places (253 → 292)
   • Ratio ambulatoire/lits: 17% → 25%
   • Stabilisation capacités 2023-2024
   • Taux occupation remonte: 81% en 2024

🔮 IMPLICATIONS POUR LE PILOTAGE:
   • Anticiper les pics épidémiques (grippe, COVID)
   • Optimiser le taux d'occupation (cible 85%)
   • Développer l'ambulatoire pour compenser fermetures
   • Maintenir capacité soins critiques flexible
""")